# Machine Learning

Una vez realizados los análisis y tranformaciones de la data seguimos por acá, para aplicar el modelo de Machine Learning.

Decidí volcarme por usar una regresión de LightGMB, que es un modelo con una buena reputación.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
df = pd.read_csv('data_ml.csv')

Vamos a empezar a trabajar con el modelo de ML!!

Primero separamos la variable predecir.

In [4]:
X = df.drop('price', axis=1)
y = df['price']

Instanciamos el modelo

In [5]:
regressor = LGBMRegressor()

Creamos la grilla de parámetros.

In [6]:
param_grid = {"n_estimators" : [210, 220, 230],
             "learning_rate" : [0.21, 0.22, 0.24,],
              "max_depth" : [4, 5, 6]}

### Hiperparámetros a explorar:

- learning_rate: La tasa de aprendizaje controla cuánto se ajustan los valores de los hiperparámetros en cada paso de optimización. Un valor menor puede ayudar a mejorar la convergencia, pero también podría requerir más iteraciones.

- n_estimators: El número de árboles en el modelo. Puede aumentar la capacidad del modelo al agregar más árboles, pero también aumentará el tiempo de entrenamiento.

- max_depth: La profundidad máxima de cada árbol. Puede controlar la complejidad del modelo y prevenir el sobreajuste.

- min_child_samples: El número mínimo de muestras en un nodo hoja. Puede controlar la regularización y evitar divisiones en nodos con muy pocas muestras.

- subsample: La fracción de muestras utilizadas para entrenar cada árbol. Puede controlar la varianza y prevenir el sobreajuste.

- colsample_bytree o colsample_bylevel: La fracción de características utilizadas para entrenar cada árbol o nivel. Puede controlar la varianza y mejorar la diversidad de árboles.

- reg_alpha y reg_lambda: Parámetros de regularización L1 y L2, respectivamente. Pueden ayudar a evitar el sobreajuste y controlar la complejidad del modelo.

- min_split_gain: Ganancia mínima requerida para realizar una división en un nodo. Puede controlar la cantidad de mejora requerida para agregar un nodo adicional.

- num_leaves: Número máximo de hojas en cada árbol. Puede controlar la complejidad del modelo y la capacidad de adaptación a los datos.

- scale_pos_weight: Utilizado en problemas de clasificación desequilibrados para ajustar el peso de las clases positivas y negativas.

Instanciamos el método GridSearchCV.

In [7]:
grid_search = GridSearchCV(regressor, param_grid, scoring='neg_mean_squared_error', cv=5)

Fiteamos el modelo.

In [8]:
grid_search.fit(X, y)

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.21, 0.22, 0.24],
                         'max_depth': [4, 5, 6],
                         'n_estimators': [210, 220, 230]},
             scoring='neg_mean_squared_error')

Vemos los hiiperparámetros que obtuvieron un mejor resultado en el modelo.

In [9]:
best_params = grid_search.best_params_
best_params

{'learning_rate': 0.22, 'max_depth': 5, 'n_estimators': 220}

In [10]:
best_mse = grid_search.best_score_
rmse = np.sqrt(-best_mse)

Claculamos el RMSE mas bajo obtenido

In [11]:
rmse

8.14425730002838

Entrenamos un modelo con los mejores hiperparámetros, para luego poder aplicarlo en el deploy.

In [12]:
lgbmr = LGBMRegressor(learning_rate=0.22, max_depth=5, n_estimators=220)

In [13]:
lgbmr.fit(X, y)

LGBMRegressor(learning_rate=0.22, max_depth=5, n_estimators=220)

Guardamos el modelo como archivo **pickle** para poder pasarlo al main, sin tener que entrenar el modelo con cada consulta.

<p><img src="src/03_pickle.png" height=200> <p>

In [14]:
with open('lgbm_regressor_model.pkl', 'wb') as model:
    pickle.dump(lgbmr, model)

Hacemos la prueba de levantar el modelo de LGBMRegression, y lo llamamos con otro nombre.

In [15]:
with open('lgbm_regressor_model.pkl', 'rb') as modelo:
        modelo_lgbm = pickle.load(modelo)

Podemos ver la cantidad de features usadas por este modelo, ya que lo guardamos ya entrenado.

In [16]:
print(modelo_lgbm.n_features_)

380


In [17]:
prediccion = modelo_lgbm.predict(X.sample(1, random_state=17))

Vemos una predicción y el RMSE de esa predicción.

In [18]:
f"Precio predicho: {prediccion[0]}"

'Precio predicho: 4.495673579571861'

In [19]:
rmse = np.sqrt(mean_squared_error(y.sample(1, random_state=17), prediccion))

print("RMSE:", rmse)

RMSE: 1.5056735795718605


Vamos a preparar un df con algunos valores seteados en 0, para luego pasarle los datos para hacer la predicción.

In [20]:
x_prediccion = X.head(1)
lista_columnas = X.columns.to_list()
x_prediccion

,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [21]:
lista_columnas[:17]

['early_access',
 'sentiment',
 'publisher_cat',
 'developer_cat',
 'year',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 "1990's"]

In [22]:
lista_features = lista_columnas[16:]
lista_features[0]

"1990's"

In [23]:
x_prediccion.loc[:,lista_features] = 0
x_prediccion

c:\Users\cristian_torres\.conda\envs\myenv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [24]:
x_prediccion.columns = [column.lower() for column in x_prediccion.columns]
x_prediccion

,early_access,sentiment,publisher_cat,developer_cat,year,month_2,month_3,month_4,month_5,month_6,...,warhammer 40k,web publishing,werewolves,western,word game,world war i,world war ii,wrestling,zombies,e-sports
0,False,0.0,1,1,2018,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


Por último también guardamos una fila vacía para usar como base y no tener que hacer el proceso en el main.

In [25]:
with open('x_prediccion.pkl', 'wb') as x_pred:
    pickle.dump(x_prediccion, x_pred)